In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
from numba import njit
import pandas as pd
import re

In [2]:
fp = 'data_000000.dat'

m = np.fromfile(fp, dtype = '>u8') # default: '<u8'

mi = m#.astype(int) # non serve, anzi meglio evitarlo così resta tutto unsigned

mat = np.zeros((mi.shape[0], 6), dtype = "u4") # orbit raggiunge i 32 bit

mat[:,0] = mi & int('1' * 5, 2) # tdc, 0-4 (5)
mat[:,1] = (mi & int('1' * 12 + '0' * 5, 2)) >> 5 # bx, 5-16 (12)
mat[:,2] = (mi & int('1' * 32 + '0' * 17, 2)) >> 17 # orbit, 17-48 (32)
mat[:,3] = (mi & int('1' * 9 + '0' * 49 , 2)) >> 49 # chan, 49-57 (9)
mat[:,4] = (mi & int('1' * 3 + '0' * 58, 2)) >> 58 # fpga (3)
#mat[:,5] = (mi & int('1' * 3 + '0' * 61, 2)) >> 61 # head (3)
# la riga sopra fallisce a causa di un overflow, sarebbe un bitwise di un numero con un array troppo "stretto" rispetto all'altro numero
# meglio prima traslare e poi fare l'and
mat[:,5] = mi >> 61 # del resto basta questo, è ovvio che se tolgo gli ultimi 61 bits restano solo i primi 3

#print(mat[1])
#print(mat.dtype)
df_big = pd.DataFrame(mat, columns = ['tdc', 'bx', 'orbit', 'chan', 'fpga', 'head'])
display(df_big)

,tdc,bx,orbit,chan,fpga,head
0,4,1609,1098099315,0,0,0
1,4,1608,586394227,382,0,0
2,4,1612,7580275,476,5,4
3,4,1613,2155063923,40,4,5
4,0,3258,3012945097,310,0,2
...,...,...,...,...,...,...
1310587,0,1714,796284928,283,1,6
1310588,0,0,4292491436,511,0,4
1310589,0,4,5912748,277,4,3
1310590,0,5,2153396396,107,2,1


In [3]:
m = np.fromfile(fp, dtype = '<u8') # default: '<u8' che sarebbe la little endianess ovvero i bits significativi si trovano a destra della riga estratta dallo stream di dati. Inoltre per determinare la lunghezza di ciascuna riga all'interno di dtype è specificato  il  dtype='u8'. Poichè le cpu sono in grado di leggere un file binario per bytes allora si richiede una riga letta alla volta che sia lunga 8 bytes cioè 64 bits come avremmo voluto inizialmente. Inoltre per convertire successivamente la riga letta tramite i singoli bits, dunque con operazioni bitwise, nelle informazioni delle hits rivelate bisogna leggere tali stringhe come unsigned int e successivamente riconvertirle in singoli bits, dunque zeri e uni. Questo accade perchè vogliamo analizzare ciascuna riga per bit ma per farlo dobbiamo passare necessariamente per i bytes visto che le cpu leggono a multipli di 8 bits. 

mi = m#.astype(int) # non serve, anzi meglio evitarlo così resta tutto unsigned

mat = np.zeros((mi.shape[0], 6), dtype = "u4") # orbit raggiunge i 32 bit

mat[:,0] = mi & int('1' * 5, 2) # tdc, 0-4 (5)
mat[:,1] = (mi & int('1' * 12 + '0' * 5, 2)) >> 5 # bx, 5-16 (12)
mat[:,2] = (mi & int('1' * 32 + '0' * 17, 2)) >> 17 # orbit, 17-48 (32)
mat[:,3] = (mi & int('1' * 9 + '0' * 49 , 2)) >> 49 # chan, 49-57 (9)
mat[:,4] = (mi & int('1' * 3 + '0' * 58, 2)) >> 58 # fpga (3)
#mat[:,5] = (mi & int('1' * 3 + '0' * 61, 2)) >> 61 # head (3)
# la riga sopra fallisce a causa di un overflow, sarebbe un bitwise di un numero con un array troppo "stretto" rispetto all'altro numero
# meglio prima traslare e poi fare l'and
mat[:,5] = mi >> 61 # del resto basta questo, è ovvio che se tolgo gli ultimi 61 bits restano solo i primi 3

#print(mat[1])
#print(mat.dtype)
df_little = pd.DataFrame(mat, columns = ['tdc', 'bx', 'orbit', 'chan', 'fpga', 'head'])
display(df_little)

,tdc,bx,orbit,chan,fpga,head
0,0,0,4079645633,100,1,1
1,2,4064,4079645602,100,1,0
2,23,1476,4079645568,100,1,4
3,16,653,4079645568,100,1,5
4,2,2922,3387200947,75,0,2
...,...,...,...,...,...,...
1310587,6,438,5797423,107,0,2
1310588,3,4092,741923455,0,0,0
1310589,18,339,741923328,0,0,4
1310590,8,1721,741923328,0,0,5


In [4]:
print(df_big['head'].max())
print(df_little['head'].max())
# df_big['head'].hist();

7
5


Quello giusto è il little cioè quello che ho letto con '<u8', perché Pazzini dice che la colonna head va da 0 a 5 e solo df_little soddisfa questo requisito

In [5]:
df_big.to_csv("df_big.csv")
df_little.to_csv("df_little.csv")

In [6]:
v = np.array([5, 12, 32, 9, 3, 3])
print(v.sum()) # deve essere 64
print(v.cumsum()) # sono i numeri che metto sopra riguardo i bits che butto

64
[ 5 17 49 58 61 64]
